In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

# Load the dataset
file_path = 'phase2_only_records.csv'
data = pd.read_csv(file_path)

# Function to create sequences for CNN-LSTM model
def create_sequences(data, seq_length=24):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i+seq_length][['Demand_kWh', 'Temperature']].values
        target = data.iloc[i+seq_length]['Demand_kWh']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Define CNN-LSTM model
def create_cnn_lstm_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        LSTM(50, activation='relu', return_sequences=True),
        LSTM(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Preprocess the data
def preprocess_data(data):
    # Ensure the data is sorted by ID and Date
    data = data.sort_values(by=['From']).reset_index(drop=True)
    
    # Normalize the temperature columns only
    temp_columns = ['Temperature']
    data[temp_columns] = (data[temp_columns] - data[temp_columns].mean()) / data[temp_columns].std()

    return data

# Split the data into train and test sets
def split_data(data, test_size=0.5):
    train_data = {}
    test_data = {}

    ids = data['ID'].unique()
    
    for id_ in ids:
        id_data = data[data['ID'] == id_]
        train, test = train_test_split(id_data, test_size=test_size, shuffle=False)
        train_data[id_] = train
        test_data[id_] = test
    
    return train_data, test_data

# Preprocess the dataset
data = preprocess_data(data)

# Split the data into train and test sets
train_data, test_data = split_data(data)

# Initialize lists to store results
results = []

# Train and evaluate model for each ID
for id_ in train_data.keys():
    # Prepare training and testing data
    train_sequences, train_targets = create_sequences(train_data[id_])
    test_sequences, test_targets = create_sequences(test_data[id_])
    
    # Reshape input data for the CNN-LSTM model
    input_shape = (train_sequences.shape[1], train_sequences.shape[2])
    train_sequences = train_sequences.reshape((train_sequences.shape[0], train_sequences.shape[1], train_sequences.shape[2]))
    test_sequences = test_sequences.reshape((test_sequences.shape[0], test_sequences.shape[1], test_sequences.shape[2]))
    
    # Create and train the model
    model = create_cnn_lstm_model(input_shape)
    model.fit(train_sequences, train_targets, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
    
    # Predict on the test data
    predictions = model.predict(test_sequences)
    
    # Store results
    for i in range(len(test_targets)):
        results.append([id_, test_data[id_].iloc[i + 24]['Date'], test_data[id_].iloc[i + 24]['Hour'], test_targets[i], predictions[i][0]])

# Save predictions to a CSV file
results_df = pd.DataFrame(results, columns=['ID', 'Date', 'Hour', 'Actual_Demand', 'Predicted_Demand'])
results_df.to_csv('lstm_results_2_5050.csv', index=False)

# Calculate overall metrics
overall_mse = mean_squared_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_mae = mean_absolute_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
overall_rmse = math.sqrt(overall_mse)

print(f"Overall Test Loss (MSE): {overall_mse}")
print(f"Overall Test Loss (MAE): {overall_mae}")
print(f"Overall Test Loss (RMSE): {overall_rmse}")


FileNotFoundError: [Errno 2] No such file or directory: 'phase2_only_records.csv'